In [5]:
import pandas as pd

train_df = pd.read_csv('/home/parker/Documents/Github/mtg/metric_records_df.csv', index_col=0)
train_df

,top1,top2,top3
0,0.258070,0.397155,0.506109
1,0.245985,0.412997,0.534821
2,0.249953,0.410790,0.540025
3,0.253735,0.400534,0.548009
4,0.249775,0.428061,0.571420
...,...,...,...
7325,0.450816,0.673284,0.792241
7326,0.440853,0.660524,0.790366
7327,0.459954,0.664989,0.792030
7328,0.442232,0.656751,0.811575


In [ ]:
import plotly.express as px

px.line(train_df)
# I don't think this is right, need to manually verify the model


In [4]:
valid_df = pd.read_csv('/home/parker/Documents/Github/mtg/validation_metric_records_df.csv',  index_col=0)
valid_df

""
